## Part 2: Supervised Learning Model

Now that you've found which parts of the population are more likely to be customers of the mail-order company, it's time to build a prediction model. Each of the rows in the "MAILOUT" data files represents an individual that was targeted for a mailout campaign. Ideally, we should be able to use the demographic information from each individual to decide whether or not it will be worth it to include that person in the campaign.

The "MAILOUT" data has been split into two approximately equal parts, each with almost 43 000 data rows. In this part, you can verify your model with the "TRAIN" partition, which includes a column, "RESPONSE", that states whether or not a person became a customer of the company following the campaign. In the next part, you'll need to create predictions on the "TEST" partition, where the "RESPONSE" column has been withheld.

This notebook will be divided in several blocks

+ Data load
+ Data cleaning
+ Feature engineering
+ Data visualization
+ Model training
+ Model selection
+ Model tuning
+ Save results


In [1]:
#Import general libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import io

#Import SageMaker libraries
import boto3
import sagemaker

from sagemaker.content_types import CONTENT_TYPE_CSV

### 2.1 Data load

In [2]:
# boto3 client to get S3 data
s3_client = boto3.client('s3')
bucket_name='sagemaker-eu-west-1-848439228145'

In [3]:
import re 

# get a list of objects in the bucket
obj_list=s3_client.list_objects(Bucket=bucket_name)

def filter_csv(string):
    return re.search(r'.csv', string)


files=[]
for contents in obj_list['Contents']:
    files.append(contents['Key'])
    
filtered_list = list(filter(filter_csv, files))
    
    
# print csv objects in in S3 bucket  
print(filtered_list)

['Capstone/Udacity_AZDIAS_052018.csv', 'Capstone/Udacity_CUSTOMERS_052018.csv', 'Capstone/Udacity_MAILOUT_052018_TEST.csv', 'Capstone/Udacity_MAILOUT_052018_TRAIN.csv', 'arvato/azdias.csv', 'arvato/customers.csv', 'arvato/transform/pca/transform/test/azdias.csv.out', 'arvato/transform/pca/transform/test/customers.csv.out', 'test/customers.csv.out']


In [4]:
def load_dataframe_from_s3(s3_client, bucket, name):
    data_object = s3_client.get_object(Bucket=bucket, Key=name)
    data_body = data_object["Body"].read()
    data_stream = io.BytesIO(data_body)
    
    return pd.read_csv(data_stream, header=0, delimiter=",") 

In [5]:
mailout_train_df = None
mailout_train_df = load_dataframe_from_s3(s3_client, bucket_name, filtered_list[3])
mailout_train_df.head()

/home/ec2-user/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2903: DtypeWarning: Columns (19,20) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


,Unnamed: 0,LNR,AGER_TYP,AKT_DAT_KL,ALTER_HH,ALTER_KIND1,ALTER_KIND2,ALTER_KIND3,ALTER_KIND4,ALTERSKATEGORIE_FEIN,...,VK_DHT4A,VK_DISTANZ,VK_ZG11,W_KEIT_KIND_HH,WOHNDAUER_2008,WOHNLAGE,ZABEOTYP,RESPONSE,ANREDE_KZ,ALTERSKATEGORIE_GROB
0,0,1763,2,1.0,8.0,NaN,NaN,NaN,NaN,8.0,...,5.0,2.0,1.0,6.0,9.0,3.0,3,0,2,4
1,1,1771,1,4.0,13.0,NaN,NaN,NaN,NaN,13.0,...,1.0,2.0,1.0,4.0,9.0,7.0,1,0,2,3
2,2,1776,1,1.0,9.0,NaN,NaN,NaN,NaN,7.0,...,6.0,4.0,2.0,NaN,9.0,2.0,3,0,1,4
3,3,1460,2,1.0,6.0,NaN,NaN,NaN,NaN,6.0,...,8.0,11.0,11.0,6.0,9.0,1.0,3,0,2,4
4,4,1783,2,1.0,9.0,NaN,NaN,NaN,NaN,9.0,...,2.0,2.0,1.0,6.0,9.0,3.0,3,0,1,3


Separate features from target

In [6]:
mailout_train_df_target = mailout_train_df['RESPONSE']

In [7]:
mailout_train_df.drop('RESPONSE', axis = 1, inplace = True)

In [8]:
mailout_test_df = None
mailout_test_df = load_dataframe_from_s3(s3_client, bucket_name, filtered_list[2])
mailout_test_df.head()

/home/ec2-user/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2903: DtypeWarning: Columns (19,20) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


,Unnamed: 0,LNR,AGER_TYP,AKT_DAT_KL,ALTER_HH,ALTER_KIND1,ALTER_KIND2,ALTER_KIND3,ALTER_KIND4,ALTERSKATEGORIE_FEIN,...,VHN,VK_DHT4A,VK_DISTANZ,VK_ZG11,W_KEIT_KIND_HH,WOHNDAUER_2008,WOHNLAGE,ZABEOTYP,ANREDE_KZ,ALTERSKATEGORIE_GROB
0,0,1754,2,1.0,7.0,NaN,NaN,NaN,NaN,6.0,...,4.0,5.0,6.0,3.0,6.0,9.0,3.0,3,1,4
1,1,1770,-1,1.0,0.0,NaN,NaN,NaN,NaN,0.0,...,1.0,5.0,2.0,1.0,6.0,9.0,5.0,3,1,4
2,2,1465,2,9.0,16.0,NaN,NaN,NaN,NaN,11.0,...,3.0,9.0,6.0,3.0,2.0,9.0,4.0,3,2,4
3,3,1470,-1,7.0,0.0,NaN,NaN,NaN,NaN,0.0,...,2.0,6.0,6.0,3.0,NaN,9.0,2.0,3,2,4
4,4,1478,1,1.0,21.0,NaN,NaN,NaN,NaN,13.0,...,1.0,2.0,4.0,3.0,3.0,9.0,7.0,4,2,4


### 2.1 Data cleaning

Find rows with more that 25% missing values

In [9]:
rows = mailout_train_df.shape[0]
missing_mailout_train = mailout_train_df.isnull().sum().sort_values(ascending = False).divide(other = (rows/100))

display(missing_mailout_train.loc[missing_mailout_train > 25])

ALTER_KIND4     99.904567
ALTER_KIND3     99.594991
ALTER_KIND2     98.240305
ALTER_KIND1     95.372655
KK_KUNDENTYP    58.926493
EXTSEL992       37.121177
dtype: float64

In [10]:
rows = mailout_test_df.shape[0]
missing_mailout_test = mailout_test_df.isnull().sum().sort_values(ascending = False).divide(other = (rows/100))

display(missing_mailout_test.loc[missing_mailout_test > 25])

ALTER_KIND4     99.908949
ALTER_KIND3     99.530736
ALTER_KIND2     98.220998
ALTER_KIND1     95.300353
KK_KUNDENTYP    58.447926
EXTSEL992       36.908458
dtype: float64

#### 2.1.1 Drop columns with more than a 60% of missing values

In [11]:
#make a dict with the names of the columns and then drop this columns from dataframe
def dropMissingColumns(df, threshold = 20):
    missing_percentages = df.isnull().sum().sort_values(ascending = False).divide(other = (rows/100))
    drop_columns = missing_percentages[missing_percentages > threshold]
    df.drop(columns = list(drop_columns.index), axis = 1, inplace = True)

In [12]:
dropMissingColumns(mailout_train_df, threshold = 60)
mailout_train_df.shape

(42962, 363)

In [13]:
dropMissingColumns(mailout_test_df, threshold = 60)
mailout_test_df.shape

(42833, 363)

#### 2.1.2 Drop low dispersion cols

In [14]:
def getVariances(df):
    df_description = df.describe()
    std_df = df_description.loc[['std']].values.reshape(df_description.shape[1],)
    return pd.Series(std_df, index =df_description.columns) 

def dropLowVarianceCols(df, threshold = 0.5):
    std_serie = getVariances(df)

    drop_lowdispersion_cols = std_serie[std_serie < threshold]
    print("Dropping columns: ",drop_lowdispersion_cols)
    df.drop(columns = list(drop_lowdispersion_cols.index), axis = 1, inplace = True)
    
    return drop_lowdispersion_cols.index

In [15]:
mailout_train_drops = dropLowVarianceCols(mailout_train_df, threshold = 0.5)

Dropping columns:  ANZ_HH_TITEL                 0.380335
ANZ_KINDER                   0.395995
ANZ_TITEL                    0.104587
D19_BANKEN_ANZ_12            0.454065
D19_TELKO_ANZ_12             0.273082
D19_TELKO_ANZ_24             0.368817
D19_TELKO_ONLINE_DATUM       0.251425
D19_TELKO_ONLINE_QUOTE_12    0.326480
D19_VERSI_ANZ_12             0.422899
D19_VERSI_ONLINE_DATUM       0.258214
D19_VERSI_ONLINE_QUOTE_12    0.395185
DSL_FLAG                     0.138167
GREEN_AVANTGARDE             0.448932
HH_DELTA_FLAG                0.335117
KBA13_KRSSEG_KLEIN           0.289578
KONSUMZELLE                  0.393734
NATIONALITAET_KZ             0.465673
SOHO_KZ                      0.097854
TITEL_KZ                     0.128331
UNGLEICHENN_FLAG             0.257269
ANREDE_KZ                    0.490881
dtype: float64


In [16]:
mailout_test_df.drop(mailout_train_drops, axis = 1, inplace = True)
mailout_test_df.shape

(42833, 342)

#### 2.1.3 Drop columns with all its values unique

In [17]:
def dropColumnsWithUniqueValues(df):
    rows = df.shape[0]
    df = df.loc[:, ( (df.nunique()/rows) < 1.0)]
            
    return df

In [18]:
print(mailout_train_df.shape)
mailout_train_df = dropColumnsWithUniqueValues(mailout_train_df)
print(mailout_train_df.shape)

(42962, 342)
(42962, 340)


In [19]:
mailout_test_df = dropColumnsWithUniqueValues(mailout_test_df)
print(mailout_test_df.shape)

(42833, 340)


#### 2.1.4 Drop rows with more than 290 missing values

In [20]:
mailout_train_df.dropna(thresh=290, inplace = True)
mailout_train_df.shape

(34214, 340)

In [21]:
mailout_test_df.dropna(thresh=290, inplace = True)
mailout_test_df.shape

(34177, 340)

Reset index after dropping rows

In [22]:
mailout_train_df.reset_index(inplace = True)

In [23]:
mailout_test_df.reset_index(inplace = True)

#### 2.1.3 Impute missing values 

In [24]:
mailout_train_df.select_dtypes(exclude=['float', 'int']).head()

,CAMEO_DEU_2015,CAMEO_DEUG_2015,CAMEO_INTL_2015,D19_LETZTER_KAUF_BRANCHE,EINGEFUEGT_AM,OST_WEST_KZ
0,5D,5,34,D19_UNBEKANNT,1992-02-10 00:00:00,W
1,5B,5,32,D19_TELKO_MOBILE,1997-05-14 00:00:00,W
2,2D,2,14,D19_LEBENSMITTEL,1995-05-24 00:00:00,O
3,2D,2,14,D19_UNBEKANNT,1992-02-10 00:00:00,W
4,7B,7,41,D19_BEKLEIDUNG_GEH,1992-02-10 00:00:00,W


There are two columns that supposedly contain numeric values but their dtype is not float or int, so they are suspicious to have categorical values instead of nan, let's find out

In [25]:
print(mailout_train_df['CAMEO_INTL_2015'].unique())
print(mailout_train_df['CAMEO_DEUG_2015'].unique())

[34.0 32.0 14.0 41.0 24.0 33.0 25.0 31.0 22.0 43.0 13.0 55.0 23.0 54.0
 51.0 45.0 12.0 44.0 35.0 15.0 52.0 nan '23' '44' '14' '55' '51' '45' '43'
 '22' '54' '24' '25' '13' '12' '35' '33' '41' '15' '52' '32' '34' '31'
 'XX']
[5.0 2.0 7.0 4.0 3.0 6.0 1.0 8.0 9.0 nan '4' '6' '2' '9' '8' '7' '3' '1'
 '5' 'X']


We can see that they have XX and X when it should be a nan value

In [26]:
def replaceForNan(df):
    df['CAMEO_INTL_2015'].replace('XX',np.nan,inplace = True)
    df['CAMEO_DEUG_2015'].replace('X',np.nan, inplace = True)

In [27]:
replaceForNan(mailout_train_df)

In [28]:
mailout_train_df['CAMEO_DEUG_2015'] = mailout_train_df['CAMEO_DEUG_2015'].astype('float32')
mailout_train_df['CAMEO_INTL_2015'] = mailout_train_df['CAMEO_INTL_2015'].astype('float32')

Also it is necessary to convert from timestamp to int the feature EINGEFUEGT_AM

In [29]:
def timestampToInt(df, column):
    timestamp =  pd.to_datetime(df[column]) ## pandas recognizes your format

    df[column] = timestamp.dt.strftime('%Y%m%d')
    return mailout_train_df[column].astype('int32')

In [30]:
mailout_train_df['EINGEFUEGT_AM'] = timestampToInt(mailout_train_df, 'EINGEFUEGT_AM')
mailout_train_df['EINGEFUEGT_AM'] = mailout_train_df['EINGEFUEGT_AM'].astype('int')

In [31]:
mailout_train_df.select_dtypes(exclude=['float', 'int', 'float32']).head()

,CAMEO_DEU_2015,D19_LETZTER_KAUF_BRANCHE,OST_WEST_KZ
0,5D,D19_UNBEKANNT,W
1,5B,D19_TELKO_MOBILE,W
2,2D,D19_LEBENSMITTEL,O
3,2D,D19_UNBEKANNT,W
4,7B,D19_BEKLEIDUNG_GEH,W


In [32]:
numerical_mailout_train = mailout_train_df.select_dtypes(include=['float', 'int','float32'])

Impute numerical columns

In [33]:
%%time

from sklearn.experimental import enable_iterative_imputer

from sklearn.impute import IterativeImputer

imp_mean = IterativeImputer(random_state=0)
imputed_mailout_train = imp_mean.fit_transform(numerical_mailout_train)

CPU times: user 22min 52s, sys: 4min 39s, total: 27min 31s
Wall time: 7min 48s


In [34]:
mailout_train_df.update(pd.DataFrame(imputed_mailout_train, columns = numerical_mailout_train.columns))

Impute categorical columns

In [39]:
def impute_mode_categorical(df):
    categorical_columns= df.select_dtypes(include=['O'])
    cols = list(df)
    
    for column in categorical_columns: 
        col_data = df[column]
        
        col_data.replace(-1,np.nan, inplace = True)
        null_data = sum(col_data.isna())
        mode = col_data.mode()[0]
        if null_data > 0:
            col_data.fillna(mode, inplace=True)
            
    return df

In [40]:
mailout_train_df.update(impute_mode_categorical(mailout_train_df.select_dtypes(exclude=['float', 'int', 'float32'])))

/home/ec2-user/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/pandas/core/generic.py:6586: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)
/home/ec2-user/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/pandas/core/generic.py:6130: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


#### 2.1.4 One hot encoding

In [49]:
mailout_train_df.select_dtypes(include=['O']).columns

Index(['CAMEO_DEU_2015', 'D19_LETZTER_KAUF_BRANCHE', 'OST_WEST_KZ'], dtype='object')

In [50]:
mailout_train_df = pd.get_dummies(mailout_train_df, columns =mailout_train_df.select_dtypes(include=['O']).columns)

### 2.3 Feature engineering

### 2.4 Data visualization

### 2.5 Model training